# Setting up a Transfer Learning Architecture for Image Classification

Transfer learning is a powerful approach in deep learning, enabling models to leverage knowledge from large datasets and apply it to specific tasks with relatively smaller datasets. This method is especially beneficial in scenarios where data collection and labeling are expensive or time-consuming.

Transfer learning architecture can handle  classification of multiple items in an image.Transfer learning involves taking a pre-trained model (usually trained on a large dataset like ImageNet, which contains millions of images across thousands of categories) and repurposing it for a different but related problem.

Options for pre-trained models include: ResNet, VGG, MobileNet, Inception and others.

For classifying multiple items within a single image, we will frame the problem as a multi-label classification problem. This means the model can predict the presence of multiiple items in an image, not just a single dominant object. In this case, we will seek to identify the presence of four states of brain status: no tumor, glioma tumor, meningioma tumor, and pituitary tumor.

The last layer of the pre-trained model will be replaced by a new fully connected layer with four outputs.

The activation function of the output layer will be sigmoid, which treats each output as a probability and independently.  This allows for the possibility of multiple items (up to four) can be identified in a single image.

The model will be trained to output a vector of probabilities for each item. All vectors will be contained in the final layer as a new fully connected layer with four outputs.

The model will be trained to maximize the probability of the correct item.


# Section 1  Prepare environment

In [1]:
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
from PIL import Image  # For handling images

import zipfile  # For handling ZIP files
import os  # For handling directories

# Import necessary libraries from Keras.
from tensorflow.keras.applications import VGG16  # For loading the VGG16 model pre-trained on ImageNet dataset
from tensorflow.keras import layers, models  # For creating custom layers and models
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For image data augmentation and preprocessing


# Section 2 Load and preprocess image dataset



Make .zip image dataset accessible for preprocessing.

In [ ]:
# Create an upload-the-dataset-button in your notebook.
from google.colab import files
uploaded = files.upload()

In [ ]:
# Unzip contents of zip file into Colab Notebook.

# Define the path to the ZIP file. 'archive.zip' is the name of the ZIP file
# that you uploaded into your Colab Notebook.
zip_path = 'archive.zip'

# Extract the contents of the ZIP file into current working director
# of Colab Notebook.
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('.')

# Clean up by removing the ZIP file.
os.remove(zip_path)

# Check if the extraction was successful
!tree

Access images

In [ ]:
# ************ O P T I O N A L *****************
# # Create an instance of ImageDataGenerator
# datagen = ImageDataGenerator(rescale=1./255)

# # Specify the directory where your images are located
# image_directory = 'archive'

# # Load the images from the directory
# train_generator = datagen.flow_from_directory(
#     image_directory,
#     target_size=(150, 150),
#     batch_size=32,
#     class_mode='binary'
# )

# # Print the class indices
# print(train_generator.class_indices)

# # Print the class names
# print(train_generator.class_indices.keys())

# # Print the number of classes
# print(len(train_generator.class_indices))

# # Print the shape of the image
# print(train_generator.image_shape)

# # Print the shape of the labels
# print(train_generator.labels.shape)


Actual loading of image files for training and validation are handled by 'ImageDataGenerator' class. This class loads images from directories, applyies data preprocessing and augmentation, and serves them to the model in batches.

In [ ]:
# Load the training and validation data generators.

# Data augmentation and preprocessing parameters can be adjusted
# to improve model generalization. In this setup, train_generator and
# validation_generator are responsible for loading images from their
# respective directories, applying specified preprocessing and augmentation
# transformations, and providing the images in batches to the model during
# the training and validation phases

# Initialize the ImageDataGenerator for the training data with several data augmentation parameters.
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescales the image by dividing pixel values by 255 (normalization to 0-1 range).
    rotation_range=40,  # Randomly rotates images within a range of 40 degrees.
    width_shift_range=0.2,  # Randomly shifts images horizontally by up to 20% of the image width.
    height_shift_range=0.2,  # Randomly shifts images vertically by up to 20% of the image height.
    shear_range=0.2,  # Applies shear transformation with an intensity of 20%.
    zoom_range=0.2,  # Randomly zooms inside pictures by up to 20%.
    horizontal_flip=True,  # Randomly flips images horizontally.
    fill_mode='nearest'  # Uses the nearest pixel values to fill gaps that may occur during rotation or shifting.
)

# Initialize the ImageDataGenerator for the validation data.
# Typically, we only rescale the validation data without applying any data augmentation.
validation_datagen = ImageDataGenerator(rescale=1./255  # Rescales the image by dividing pixel values by 255 (normalization to 0-1 range).
)

train_generator = train_datagen.flow_from_directory(
    'archive/train',
    target_size=(224, 224),  # Match the input size expected by the model
    batch_size=32,
    class_mode='categorical'  # 'categorical' for multi-class classification
)

validation_generator = validation_datagen.flow_from_directory(
    './data/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Now, you can use these generators in model.fit()
# model.fit(train_generator, epochs=epochs, validation_data=validation_generator)


# Section 3 Configure Pretrained Model

# Section 4 Add Classification Layers

# Section 5 Compile Model

---



# Section 6 Train Model

# Section 7 Evaluate Model

# Section 8 Fine-tuning

Prepare environment

Load base model

In [ ]:
# Load the base model, VGG16, pre-trained on the ImageNet dataset.
# 'weights='imagenet'' loads the weights trained on ImageNet.
# 'include_top=False' excludes the top (fully connected) layers of the model, making it suitable for feature extraction.
# 'input_shape=(224, 224, 3)' sets the expected input size to 224x224 pixels with 3 color channels (RGB).
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Freeze base model

In [ ]:
# Freeze the base model to prevent its weights from being updated during training.
# This is done because we want to utilize the learned features without altering them.
base_model.trainable = False

Create a new model ON TOP of existing frozen pre-trainedVGG16 model

In [ ]:
# Create a new model on top of the base model.
model = models.Sequential([
    base_model,  # The pre-trained VGG16 model as the feature extractor.
    layers.GlobalAveragePooling2D(),  # Applies global average pooling to the output of the base model.
    layers.Dense(1024, activation='relu'),  # A dense layer with 1024 neurons and ReLU activation function for learning non-linearities.
    layers.Dropout(0.2),  # Dropout layer for regularization, dropping out 20% of the input units randomly to reduce overfitting.
    layers.Dense(4, activation='softmax')  # Output layer with 4 neurons (one for each class) and softmax activation for multi-class classification.
])

Compile model

In [ ]:
# Compile the model with the Adam optimizer, categorical crossentropy as the loss function
# (suitable for multi-class classification), and track the 'accuracy' metric during training.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Leverage Image Data Generator Libraryfor training and **validation**

In [ ]:
# Need below in more plain speak.
# Instantiate ImageDataGenerator for training data with data augmentation parameters.
# 'rescale=1./255' scales image pixel values to a range of 0 to 1.
# 'data_augmentation_strategy' variable is assumed to define specific data augmentation parameters (e.g., rotation, zoom),
# which is not explicitly defined in the snippet and should be replaced with actual parameters or removed.
train_datagen = ImageDataGenerator(rescale=1./255, data_augmentation_strategy) # Here, replace or remove 'data_augmentation_strategy'.

# Instantiate the ImageDataGenerator for validation data without data augmentation, only rescaling.
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Assuming 'train_generator' and 'validation_generator' are defined elsewhere in the code to load images from directories
# using the flow_from_directory method of the respective ImageDataGenerator instances.
# This involves specifying the path to the data, target image size, batch size, and class mode.


In [ ]:

# Data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, data_augmentation_strategy)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Fit the model
# model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

In [ ]:
# Create a new model on top of the base model.
model = models.Sequential([
    base_model,  # The pre-trained VGG16 model as the feature extractor.
    layers.GlobalAveragePooling2D(),  # Applies global average pooling to the output of the base model.
    layers.Dense(1024, activation='relu'),  # A dense layer with 1024 neurons and ReLU activation function for learning non-linearities.
    layers.Dropout(0.2),  # Dropout layer for regularization, dropping out 20% of the input units randomly to reduce overfitting.
    layers.Dense(4, activation='softmax')  # Output layer with 4 neurons (one for each class) and softmax activation for multi-class classification.
])

# Compile the model with the Adam optimizer, categorical crossentropy as the loss function (suitable for multi-class classification),
# and track the 'accuracy' metric during training.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Instantiate the ImageDataGenerator for training data with data augmentation parameters.
# 'rescale=1./255' scales the image pixel values to a range of 0 to 1.
# The 'data_augmentation_strategy' variable is assumed to define specific data augmentation parameters (e.g., rotation, zoom),
# which is not explicitly defined in the snippet and should be replaced with actual parameters or removed.
train_datagen = ImageDataGenerator(rescale=1./255)  # Here, replace or remove 'data_augmentation_strategy'.

# Instantiate the ImageDataGenerator for validation data without data augmentation, only rescaling.
validation_datagen = ImageDataGenerator(rescale=1./255)

# Assuming 'train_generator' and 'validation_generator' are defined elsewhere in the code to load images from directories
# using the flow_from_directory method of the respective ImageDataGenerator instances.
# This involves specifying the path to the data, target image size, batch size, and class mode.

# Train the model using the fit method, specifying the training and validation data generators, number of epochs, and other parameters.
# 'epochs=epochs' should be replaced with a specific number of epochs, for example, 'epochs=10'.
# This line is commented out and should be uncommented and completed with actual variable names and values.
# model.fit(train_generator, epochs=epochs, validation_data=validation_generator)


In [ ]:
# Print the first few image filenames
path = "https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_1/datasets/csvs/fungi_files.csv"

filenames_df = pd.read_csv(path)
filenames_df.head()

,name
0,H1_103a_3.jpg
1,H1_38b_2.jpg
2,H2_8f_14.jpg
3,H1_99a_2.jpg
4,H2_49c_2.jpg


In [ ]:
# First, remove the .jpg file extension,
# then split into three new columns: 'class', 'sample', and 'image'
filenames_df[['class', 'sample', 'image']] = filenames_df['name']\
                                                            .str.replace('.jpg', '', regex=False)\
                                                            .str.split('_', expand=True)
filenames_df.head()

,name,class,sample,image
0,H1_103a_3.jpg,H1,103a,3
1,H1_38b_2.jpg,H1,38b,2
2,H2_8f_14.jpg,H2,8f,14
3,H1_99a_2.jpg,H1,99a,2
4,H2_49c_2.jpg,H2,49c,2


In [ ]:
from google.colab import drive
import pickle
drive.mount('/content/drive')
# For our purposes, we'll select the class column as 'y'
y = filenames_df['class']

# And we'll export this as another pkl file
with open('/content/drive/My Drive/fungi_y.pkl', 'wb') as file:
    pickle.dump(y, file)